In [34]:
!pip install keras_tuner

     |████████████████████████████████| 71kB 2.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=a9963cbcb89728b890df06153efa85010a8d9c5a81f96d8f19299e0811d9a3ad
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=a3e2ea2d3426e2a7a7978ea90eac127761333eff679f2b134f1be7a07cd5894d
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [35]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [36]:
fashion_mnist=keras.datasets.fashion_mnist

In [37]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [38]:
train_images=train_images/255.0
test_images=test_images/255.0

In [39]:
train_images[0].shape

(28, 28)

In [40]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [41]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128,step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [45]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [48]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [49]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 32m 22s]
val_accuracy: 0.9131666421890259

Best val_accuracy So Far: 0.9131666421890259
Total elapsed time: 01h 09m 47s
INFO:tensorflow:Oracle triggered exit


In [50]:
model=tuner_search.get_best_models(num_models=1)[0]

In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        134448    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,738,794
Trainable params: 2,738,794
Non-trainable params: 0
_________________________________________________________________
